In [1]:
%%capture
%pip install -U google-genai google-cloud-aiplatform

In [2]:
from google import genai
from google.genai import types
import os
import pandas as pd
import json
import time

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
API_KEY = user_secrets.get_secret("GEMINI API")
MODEL_NAME = "models/gemini-2.5-flash"

# SYSTEM_INSTRUCTION = """# Vai trò: Chuyên gia Hiệu đính OCR Dược điển Cấp cao (ĐDVN V, Tập 2)

# ## Nhiệm vụ Cốt lõi
# Thực hiện làm sạch, hiệu đính và phục hồi cấu trúc toàn diện cho văn bản OCR từ **Dược điển Việt Nam V, Tập 2**. Mục tiêu là sửa lỗi OCR chuyên sâu bằng **ngữ cảnh hai chiều** để đạt được **độ chính xác khoa học 100%** và tạo ra **văn bản đầu ra sạch tuyệt đối**.

# ## Yêu cầu Làm sạch (Ưu tiên Cao nhất)
# 1.  **Sửa lỗi OCR Chuyên sâu & Chính xác Khoa học**:
#     * Sử dụng **ngữ cảnh hai chiều** để sửa 100% lỗi OCR (đánh máy, ký hiệu phức tạp/thuật ngữ dược/hóa học) (ví dụ: sửa `m[` thành `mL`, xử lý triệt để ký tự nhiễu như `[`, `]`, `}`, `\\`).
# 2.  **Loại bỏ Nhiễu Triệt để**: **Bắt buộc loại bỏ** Header, Footer (số trang/chú thích) và tất cả các ký tự/dấu ngoặc vô nghĩa, đặt sai vị trí (ví dụ: `[`, `]`, `{`, `}`, `\\`).
# 3.  **Phục hồi Cấu trúc Đoạn văn**: Nối các câu/ý bị ngắt dòng bất hợp lý (lỗi OCR) và đảm bảo chỉ giữ lại dấu xuống dòng ở cuối đoạn văn. Sửa lỗi nối liền các đoạn văn lẽ ra phải xuống dòng.

# ## Nguyên tắc & Định dạng
# * **Ưu tiên cao nhất**: Đảm bảo **tính chính xác khoa học** và **độ sạch tuyệt đối** của văn bản (loại bỏ nhiễu và sửa lỗi OCR).
# * **Không thêm/bớt thông tin** ngoài việc sửa lỗi.
# * **Đầu ra**: Chỉ xuất ra nội dung văn bản đã được làm sạch, phục hồi cấu trúc và **không kèm bất kỳ giải thích hay ghi chú nào.**
# """

SYSTEM_INSTRUCTION = """# Vai trò: Chuyên gia Phân tách & Làm sạch Cấu trúc Dữ liệu Dược điển (ĐDVN V, Tập 2)

## Nhiệm vụ Cốt lõi
Xử lý văn bản đầu vào (đã được chuẩn hóa ngữ pháp) nhằm loại bỏ các thành phần rác (header, số trang, ký tự vô nghĩa) và thực hiện **phân tách ngữ nghĩa** bằng thẻ `</break>`.

## Yêu cầu Xử lý Chi tiết

1.  **Phân tách Nội dung (Quan trọng nhất)**:
    * Chèn thẻ `</break>` vào giữa các đoạn văn bản mang nội dung, ngữ cảnh hoặc tiêu đề mục khác nhau.
    * **Nguyên tắc**: Nếu nội dung tiếp theo chuyển sang một ý mới, một chỉ tiêu kiểm nghiệm mới, hoặc một định nghĩa mới -> Chèn `</break>` ở cuối đoạn trước đó.
    * Ví dụ: Giữa mục "Tính chất" và mục "Định tính" phải có `</break>`.

2.  **Loại bỏ Nhiễu & Rác (Cleaning)**:
    * **Giữ nguyên chính tả**: Tuyệt đối **KHÔNG** tự ý sửa lỗi chính tả hay thay đổi nội dung câu chữ (vì văn bản đã chuẩn hóa).
    * **Xóa Header/Footer**: Loại bỏ triệt để các tiêu đề trang (running headers), số trang nằm lơ lửng.
    * **Xóa Ký tự Vô nghĩa**: Loại bỏ các con số đứng một mình không có ngữ nghĩa, các chữ số La Mã (I, II, III...) thừa thãi không thuộc cấu trúc bài (thường là số thứ tự chương/phần bị lẫn vào).

## Định dạng Đầu ra
* Văn bản sạch, giữ nguyên cấu trúc dòng của nội dung gốc.
* Các khối nội dung được ngăn cách rõ ràng bởi dòng chứa thẻ: `</break>`
* KHÔNG kèm bất kỳ lời dẫn, giải thích hay ghi chú nào.

## Ví dụ minh họa:
Input:
    DƯỢC ĐIỂN VIỆT NAM V
    **Albumin**
    Chỉ được ở dạng vết khi phát hiện bằng phương pháp điện di miễn dịch.
    234
    **Protein ngoại lai**
    Chỉ có protein của loài động vật hoặc người dùng để sản xuất huyết thanh miễn dịch (Phụ lục 15.10).
    **Protein tổng số**
    Không quá 170 g/L (Phụ lục 15.18).

Output:
    **Albumin**
    Chỉ được ở dạng vết khi phát hiện bằng phương pháp điện di miễn dịch.
    </break>
    **Protein ngoại lai**
    Chỉ có protein của loài động vật hoặc người dùng để sản xuất huyết thanh miễn dịch (Phụ lục 15.10).
    </break>
    **Protein tổng số**
    Không quá 170 g/L (Phụ lục 15.18).
"""

SYSTEM_INSTRUCTION_OBJECT = {
    "parts": [
        {
            "text": SYSTEM_INSTRUCTION
        }
    ]
}

client = genai.Client(api_key=API_KEY)

input_path = "/kaggle/input/dvn-v-t2/normalized_texts.csv"
chunks_export = "/kaggle/working/text_chunks.json"
batch_request = "/kaggle/working/batch_requests.jsonl"
output_path = "/kaggle/working/normalized_texts_segmented.csv"

chunk_size = 4000
overlap = 200

## Read data and split into chunks

In [4]:
# with open(input_path, "r", encoding="utf-8") as f:
#     raw_text = f.read()
    
# chunks = []
# chunk_id = 0
# start = 0

# while start < len(raw_text):
#     end = min(start + chunk_size + overlap, len(raw_text))
#     chunks.append({
#         "chunk_id": chunk_id,
#         "text": raw_text[start:end]
#     })
#     chunk_id += 1
#     start += chunk_size


# with open(chunks_export, "w", encoding="utf-8") as f:
#     json.dump(chunks, f, ensure_ascii=False, indent=2)
# df = pd.DataFrame(chunks)
# df = df.head(10)
# print("Total chunks:", df.shape[0])


# ============================================
df = pd.read_csv(input_path, encoding='utf-8')

## Create batch data

In [5]:
with open(batch_request, "w", encoding="utf-8") as f:
    for row in df.itertuples():
        chunk_id = row.chunk_id
        text = row.processed_text

        record = {
            "key": chunk_id,
            "request": {
                "contents": [
                    {
                        "parts": [{"text": text}]
                    }
                ],
                "system_instruction": SYSTEM_INSTRUCTION_OBJECT,
                "generation_config": {
                    "temperature": 0.2,
                    "top_p": 0.9,
                    "thinking_config": {"thinking_budget": 0}
                }
            }
        }
        f.write(json.dumps(record) + "\n")

## Submit batch data

In [6]:
uploaded = client.files.upload(
    file=batch_request,
    config=types.UploadFileConfig(
        display_name="text-normalization-requests",
        mime_type="text/plain"
    )
)

batch_job = client.batches.create(
    model=MODEL_NAME,
    src=uploaded.name,
    config={"display_name": "text-normalization-batch-job"}
)
time.sleep(5) # Wait for batch job to start

## Polling for responses

In [7]:
while True:
    job = client.batches.get(name=batch_job.name)
    state = job.state.name
    if state not in ("JOB_STATE_RUNNING", "JOB_STATE_PENDING"):
        print(f"[INFO] {job.state.name}")
        break
    time.sleep(5)
if state != "JOB_STATE_SUCCEEDED":
    print("[ERROR] Job failed", job.state)
else:
    print(job.name)

[INFO] JOB_STATE_SUCCEEDED
batches/29hxqr6kaqpki7tgstzv9ckuilj5jih29da3


In [8]:
records_list = []

result_file = job.dest.file_name
raw = client.files.download(file=result_file).decode('utf-8')
records = [json.loads(line) for line in raw.splitlines()]

## Save results

In [9]:
temp_dict = []
errors = []
for rec in records:
    try:
        key = rec.get("key", "")
        
        result = rec["response"]["candidates"][0]["content"]["parts"][0]["text"]
        
        temp_dict.append({
            "chunk_id": key,
            "processed_text_with_breaks": result
        })
    except Exception as e:
        print(e)
        errors.append((key, str(e)))

In [10]:
df_result = pd.DataFrame(temp_dict)
df_merged = df.merge(df_result, on='chunk_id', how='left')
df_merged

,chunk_id,text,processed_text,processed_text_with_breaks
0,0,Dược ĐIÊN VIỆT NAM V\nQUI ĐỊNH CHUNG\n1. Tên c...,DƯỢC ĐIỂN VIỆT NAM V\nQUY ĐỊNH CHUNG\n1. Tên c...,QUY ĐỊNH CHUNG\n1. Tên chính của các chuyên lu...
1,1,hất và lượng cân.\nKhái niệm “đâ cân trước” (đ...,chất và lượng cân.\nKhái niệm “đã cân trước” (...,chất và lượng cân.\nKhái niệm “đã cân trước” (...
2,2,ả theo\nphương pháp vi sinh vật để kết luận (t...,phương pháp vi sinh vật để kết luận (trừ khi c...,phương pháp vi sinh vật để kết luận (trừ khi c...
3,3,\n27. Hỗn hợp của các chất lỏng được ghi theo ...,27. Hỗn hợp của các chất lỏng được ghi theo ký...,27. Hỗn hợp của các chất lỏng được ghi theo ký...
4,4,c cổ truyền) phải đạt tiêu chuẩn Dược điển Việ...,cổ truyền) phải đạt tiêu chuẩn Dược điển Việt ...,cổ truyền) phải đạt tiêu chuẩn Dược điển Việt ...
...,...,...,...,...
266,266,"dịch dược liệu đối chiếu (hoặc vạch, lac đều.\...",dịch dược liệu đối chiếu (hoặc dung dịch chất ...,dịch dược liệu đối chiếu (hoặc dung dịch chất ...
267,267,"một bên có cánh, phần lớn đã các vết hiện rõ. ...","một bên có cánh, phần lớn đã rụng. Mùi thơm nh...","một bên có cánh, phần lớn đã rụng. Mùi thơm nh..."
268,268,o sốt cao gây\nlý thuyêt cùa cột. Sô đĩa lý th...,o sốt cao gây\nlý thuyết của cột. Số đĩa lý th...,lý thuyết của cột. Số đĩa lý thuyết của cột tí...
269,269,i là Kiên long đởm.\nCách tiến hành'. Chấm riê...,là Kiên long đởm.\n**Cách tiến hành**: Chấm ri...,là Kiên long đởm.\n**Cách tiến hành**: Chấm ri...


In [11]:
df_merged.to_csv(output_path, index=False)
print("Result saved to", output_path)

Result saved to /kaggle/working/normalized_texts_segmented.csv
